In [1]:
import pandas as pd
import numpy as np
from pandas import Series, DataFrame
import matplotlib.pyplot as plt
data_train = pd.read_csv(r"C:\Users\fx\Documents\GitHub\Titanic\titanic\train.csv")

# 数据处理
## 随机森林填上缺省值Age

In [2]:
from sklearn.ensemble import  RandomForestRegressor

def set_missing_ages(df):
    age_df =df[['Age','Fare','Parch','SibSp','Pclass']]
    
    known_age = age_df[age_df.Age.notnull()].as_matrix()
    unknown_age = age_df[age_df.Age.isnull()].as_matrix()
    
    y= known_age[:,0]
    
    X= known_age[:,1:]
    
    rfr = RandomForestRegressor(random_state=0,n_estimators=2000,n_jobs=-1)
    rfr.fit(X,y)
    
    predictedAges = rfr.predict(unknown_age[:,1::])
    
    df.loc[(df.Age.isnull()),'Age']= predictedAges
    return df,rfr

def set_Cabin_type(df):
    df.loc[(df.Cabin.notnull()),'Cabin']= "Yes"
    df.loc[(df.Cabin.isnull()),'Cabin']="No"
    return df


    
    

In [3]:
data_train,rfr = set_missing_ages(data_train)
data_train= set_Cabin_type(data_train)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  import sys


## 向量化

In [4]:
dummies_Cabin = pd.get_dummies(data_train['Cabin'],prefix = 'Cabin')

dummies_Embarked = pd.get_dummies(data_train['Embarked'],prefix = 'Embarked')

dummies_Sex = pd.get_dummies(data_train['Sex'],prefix = 'Sex')

dummies_Pclass =pd.get_dummies(data_train['Pclass'],prefix = 'Pclass')

df= pd.concat([data_train,dummies_Cabin,dummies_Embarked,dummies_Sex,dummies_Pclass],axis=1)

df.drop(['Pclass','Name','Ticket','Cabin','Embarked'],axis=1,inplace= True)

df.head()




,PassengerId,Survived,Sex,Age,SibSp,Parch,Fare,Cabin_No,Cabin_Yes,Embarked_C,Embarked_Q,Embarked_S,Sex_female,Sex_male,Pclass_1,Pclass_2,Pclass_3
0,1,0,male,22.0,1,0,7.2500,1,0,0,0,1,0,1,0,0,1
1,2,1,female,38.0,1,0,71.2833,0,1,1,0,0,1,0,1,0,0
2,3,1,female,26.0,0,0,7.9250,1,0,0,0,1,1,0,0,0,1
3,4,1,female,35.0,1,0,53.1000,0,1,0,0,1,1,0,1,0,0
4,5,0,male,35.0,0,0,8.0500,1,0,0,0,1,0,1,0,0,1


## Scale Age&Fare

In [5]:
import sklearn.preprocessing as preprocessing
scaler  =  preprocessing.StandardScaler()
#age_scale_param = scaler.fit(df['Age'])

#df['Age_scaled'] =scaler.fit_transform(df['Age'].values.reshape(-1,1),age_scale_param)
df['Age_scaled'] =scaler.fit_transform(df['Age'].values.reshape(-1,1))

#fare_scale_param = scaler.fit(df['Fare'])
df['Fare_scaled'] = scaler.fit_transform(df['Fare'].values.reshape(-1,1))
#df['Fare_scaled'] = scaler.fit_transform(df['Fare'],fare_scale_param)
df.head()


,PassengerId,Survived,Sex,Age,SibSp,Parch,Fare,Cabin_No,Cabin_Yes,Embarked_C,Embarked_Q,Embarked_S,Sex_female,Sex_male,Pclass_1,Pclass_2,Pclass_3,Age_scaled,Fare_scaled
0,1,0,male,22.0,1,0,7.2500,1,0,0,0,1,0,1,0,0,1,-0.561380,-0.502445
1,2,1,female,38.0,1,0,71.2833,0,1,1,0,0,1,0,1,0,0,0.613171,0.786845
2,3,1,female,26.0,0,0,7.9250,1,0,0,0,1,1,0,0,0,1,-0.267742,-0.488854
3,4,1,female,35.0,1,0,53.1000,0,1,0,0,1,1,0,1,0,0,0.392942,0.420730
4,5,0,male,35.0,0,0,8.0500,1,0,0,0,1,0,1,0,0,1,0.392942,-0.486337


##  建模?

In [7]:
from sklearn import linear_model

train_df = df.filter(regex='Survived|Age_.*|SibSp|Parch|Fare_.*|Cabin_*|Embarked_.*|Sex_.*|Pclass_.*')
train_np = train_df.as_matrix()

y = train_np[:,0]

X = train_np[:,1:]
print(X.shape)
clf = linear_model.LogisticRegression(C=1.0,penalty='l1',tol =1e-6)

clf.fit(X,y)

clf


(891, 14)


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l1', random_state=None, solver='warn',
          tol=1e-06, verbose=0, warm_start=False)

In [17]:
train_df.head()

,Survived,SibSp,Parch,Cabin_No,Cabin_Yes,Embarked_C,Embarked_Q,Embarked_S,Sex_female,Sex_male,Pclass_1,Pclass_2,Pclass_3,Age_scaled,Fare_scaled
0,0,1,0,1,0,0,0,1,0,1,0,0,1,-0.561380,-0.502445
1,1,1,0,0,1,1,0,0,1,0,1,0,0,0.613171,0.786845
2,1,0,0,1,0,0,0,1,1,0,0,0,1,-0.267742,-0.488854
3,1,1,0,0,1,0,0,1,1,0,1,0,0,0.392942,0.420730
4,0,0,0,1,0,0,0,1,0,1,0,0,1,0.392942,-0.486337


In [10]:
data_test = pd.read_csv(r"C:\Users\fx\Documents\GitHub\Titanic\titanic\test.csv")
data_test.loc[(data_test.Fare.isnull()),'Fare'] = 0

tmp_df = data_test[['Age','Fare','Parch','SibSp','Pclass']]
null_age = tmp_df[data_test.Age.isnull()].as_matrix()

X = null_age[:,1:]
predicetedAges =rfr.predict(X)
data_test.loc[(data_test.Age.isnull()),'Age'] = predicetedAges

data_test = set_Cabin_type(data_test)
dummies_Cabin = pd.get_dummies(data_test['Cabin'],prefix='Cabin')
dummies_Embarked = pd.get_dummies(data_test['Embarked'],prefix='Embarked')
dummies_Sex = pd.get_dummies(data_test['Sex'],prefix='Sex')
dummies_Pclass = pd.get_dummies(data_test['Pclass'],prefix='Pclass')

df_test = pd.concat([data_test,dummies_Cabin,dummies_Embarked,dummies_Sex,dummies_Pclass],axis=1)
df_test.drop(['Pclass','Name','Sex','Ticket','Cabin','Embarked'],axis=1,inplace=True)
df_test['Age_scaled'] = scaler.fit_transform(data_test['Age'].values.reshape(-1,1))
df_test['Fare_scaled'] = scaler.fit_transform(data_test['Fare'].values.reshape(-1,1))

df_test.head()


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """


,PassengerId,Age,SibSp,Parch,Fare,Cabin_No,Cabin_Yes,Embarked_C,Embarked_Q,Embarked_S,Sex_female,Sex_male,Pclass_1,Pclass_2,Pclass_3,Age_scaled,Fare_scaled
0,892,34.5,0,0,7.8292,1,0,0,1,0,0,1,0,0,1,0.307521,-0.496637
1,893,47.0,1,0,7.0000,1,0,0,0,1,1,0,0,0,1,1.256241,-0.511497
2,894,62.0,0,0,9.6875,1,0,0,1,0,0,1,0,1,0,2.394706,-0.463335
3,895,27.0,0,0,8.6625,1,0,0,0,1,0,1,0,0,1,-0.261711,-0.481704
4,896,22.0,1,1,12.2875,1,0,0,0,1,1,0,0,0,1,-0.641199,-0.416740


In [18]:
test = df_test.filter(regex='Age_.*|SibSp|Parch|Cabin_*|Fare_.*|Embarked_.*|Sex_.*|Pclass_.*')
test.head()
test.shape

(418, 14)

In [21]:
predictions  = clf.predict(test)
result = pd.DataFrame({'PassengerId':data_test['PassengerId'].as_matrix(),'Survived':predictions.astype(np.int32)})
result.to_csv(r"C:\Users\fx\Documents\GitHub\Titanic\titanic\prediction.csv",index=False)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [20]:
test.shape

(418, 14)